# Bio-inspired Algorithms
## 1-D
## Dariana Gomez

Objective Function
$$ f(x) = x^2 $$

## Initial parameters

In [99]:
# Selecting elements for tournaments
l_sel= [[5, 0, 1], [4, 2, 1], [5, 1, 0], [1, 5, 4], [0, 1, 4], [2, 1, 0], [1, 5, 3], [3, 2, 5], [0, 5, 3], [0, 4, 5], 
        [0, 2, 3], [4, 0, 3], [4, 3, 1], [5, 3, 4], [2, 4, 3], [0, 2, 3], [3, 4, 1], [4, 3, 5], [2, 4, 1], [2, 3, 1], 
        [3, 1, 2], [2, 4, 0], [4, 0, 5], [4, 3, 2]]

# Random values to check if crossover and/or mutation happen
l_rand= [0.59, 0.96, 0.64, 0.86, 0.44, 0.70, 0.78, 0.96, 0.67, 0.89, 0.46, 0.34, 0.24, 0.80, 0.77, 0.11, 0.91, 0.69, 
         0.80, 0.50, 0.60, 0.18, 0.55, 0.39, 0.68, 0.21, 0.01, 0.91, 0.78, 0.35, 0.04, 0.49, 0.43, 0.13, 0.82, 0.66, 
         0.40, 0.50, 0.93, 0.64, 0.75, 0.96, 0.13, 0.30, 0.63, 0.55, 0.46, 0.87, 0.03, 0.59, 0.87, 0.42, 0.34, 0.68, 
         0.99, 0.32, 0.15, 0.69, 0.40, 0.49, 0.58, 0.48, 0.76, 0.22, 0.73, 0.50, 0.10, 0.21, 0.95, 0.64, 0.41, 0.13, 
         0.69, 0.48, 0.41, 0.08, 0.34, 0.14, 0.62, 0.71, 0.01, 0.70, 0.05, 0.37, 0.96, 0.26, 0.58, 0.88, 0.95, 0.05, 
         0.06, 0.51, 0.39, 0.61, 0.45, 0.42, 0.25, 0.69, 0.18, 0.66, 0.72, 0.33, 0.90, 0.78, 0.36, 0.37, 0.91, 0.15, ]

# Select bit where crossover starts
l_cross= [1, 2, 2, 1, 2, 2, 2, 1, 1, 1]

# Initial population
pop = [[1, 0, 1, 0], [1, 0, 1, 0], [1, 0, 0, 0], [1, 1, 1, 0], [1, 0, 0, 0], [1, 0, 0, 1]]


# % of crossover and mutation desired
r_cross = 0.9
r_mut = 0.25

# Range for values entering objective function
min_value = -1
max_value = 1


## Mapping process

In [100]:
# Turn lists into binary values

def list2bin(population):
    binary_numbers = []

    for row in population:
        binary_number = ""
        for bit in row:
            binary_number += str(bit)
        binary_numbers.append(binary_number)
    return binary_numbers

    
# Turn binary values into decimal values   

def bin2dec(bins):
    decimal_values = []

    for binary_number in bins:
        decimal_value = int(binary_number, 2)
        decimal_values.append(decimal_value)
    return decimal_values


# Map decimal values into values to evaluate objective function


def mapping(decValues):
    decoded_values = []
    bits_ind = 4 # bits per individual
    
    for decimal_value in decValues:
        decoded_value = min_value + decimal_value*(max_value - min_value)/(2**bits_ind-1)
        decoded_values.append(decoded_value)
    return decoded_values


In [101]:
binaries = list2bin(pop)
decimals = bin2dec(binaries)
dec_vals = mapping(decimals)

## Objective function and individuals' fitnesses

In [102]:
# Evaluating objective function

obj_function = lambda x: x**2

def fitness_fcn(decoded_values):
    # Obtain the fitnesses for each individual

    fitnesses = []

    for decoded_value in decoded_values:
        fitness = obj_function(decoded_value)
        fitnesses.append(fitness)
    
    # Dictionary to help us retrieve individuals' indices ahead

    
    return fitnesses
    


In [103]:
fit_list =  fitness_fcn(dec_vals)

fit_dict = {index: value for index, value in enumerate(fitnesses)} 

fit_dict

{0: 0.11111111111111106,
 1: 0.11111111111111106,
 2: 0.004444444444444443,
 3: 0.7511111111111112,
 4: 0.004444444444444443,
 5: 0.03999999999999998}

## Tournament process

In [104]:
def tournament(generation, selection, fitnesses): 
    # Separate tournaments' list into smaller lists (same amount as individuals in initial population)

    n = 6 # Number of elements per smaller list

    tournaments= []

    smaller_lists = [selection[i:i + n] for i in range(0, len(selection), n)]

    for sublist in smaller_lists:
        tournaments.append(sublist)
        
    sorted_list = []
    comps = 3 # competitors per tournament
    
    # actual tournament process
    
    champs = []
    
    for tourns in tournaments[generation]:
        for competitor in tourns:
            sorted_list.append(fitnesses[competitor])
    smaller_tourns = [sorted_list[i:i + comps] for i in range(0, len(sorted_list), comps)]
    for element in smaller_tourns:
        champs.append(min(element))
        
    return champs

In [105]:
champs = tournament(0, l_sel, fit_list)

### Champions

In [106]:
def champs_fcn(champions):
    # Retrieve indices to know which individuals will integrate new population

    indices = [key for value in champions for key, dict_value in fit_dict.items() if value == dict_value]
    subtourns_ = [indices[i:i+6] for i in range(0, len(indices), 6)]

    subtourns = subtourns_[0]
    
    return subtourns


In [107]:
subtourns = champs_fcn(champs)

In [108]:
# Separate list of population (not sure why tbh)

sep_gen = [pop[i:i + 1] for i in range(0, len(pop), 1)]


## New Generation

In [109]:
# Create list of individuals for next generation

new_gen = []

for index in subtourns:
    new_gen.append(sep_gen[index])

## Crossover and mutation

### Crossover

In [110]:
import random

# Crossover function
def crossover(parent1, parent2, crossover_prob, crossover_points):
    if random.random() <= crossover_prob:
        child1 = parent1[:crossover_points] + parent2[crossover_points:]
        child2 = parent2[:crossover_points] + parent1[crossover_points:]
        return child1, child2
    else:
        return parent1, parent2

# Define your population and crossover parameters
population_size = 6
crossover_probability = 0.9  # You can adjust this as needed
crossover_points_list = l_cross  # Replace with your list of crossover points

# Perform crossover on the parents to generate children
children = []

for i in range(0, population_size, 2):
    if crossover_points_list:  # Check if there are remaining crossover points
        child1, child2 = crossover(parents[i][0], parents[i+1][0], crossover_probability, crossover_points_list[0])
        crossover_points_list = crossover_points_list[1:] + [crossover_points_list[0]]  # Move the used point to the end
    else:
        child1, child2 = parents[i][0], parents[i+1][0]  # No more crossover points, just copy parents

    children.append(child1)
    children.append(child2)

print(children)


[[1, 0, 0, 0], [1, 0, 0, 1], [1, 0, 0, 1], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0]]


### Mutation

In [111]:
import random

# Mutation function
def mutation(child2mut, list_values, rmut):
    chrom = 0
    mutated_child = child2mut.copy()  # Create a copy of the child to modify
    sublist_index = 0  # Initialize the index for the sublist
    
    for item in list_values:
        if sublist_index >= len(list_values[sublist_index]):
            sublist_index += 1  # Move to the next sublist if the current sublist is exhausted
            chrom = 0  # Reset the chrom index for the new sublist
            
        if sublist_index < len(list_values) and item <= rmut:
            mutated_child[chrom] = 1 - child2mut[chrom]  # Toggle 1 to 0 and vice versa
            chrom += 1
        
    return mutated_child


# Define your mutation parameters
mutation_probability = r_mut  
mutation_list = mut_valid[:len(children[0])]  # Ensure mutation_list has the same length as children

# Apply mutation to children
mutated_children = []

for child in children:
    mutated_child = mutate(child, mutation_list, mutation_probability)
    mutated_children.append(mutated_child)

print(mutated_children)


[[0, 0, 1, 0], [1, 1, 0, 0], [1, 0, 0, 1], [1, 0, 1, 0], [0, 0, 0, 0], [1, 0, 1, 0]]


In [112]:
new_pop = []
new_pop = mutated_children

print("The new population is:", new_pop)

The new population is: [[0, 0, 1, 0], [1, 1, 0, 0], [1, 0, 0, 1], [1, 0, 1, 0], [0, 0, 0, 0], [1, 0, 1, 0]]
